In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Carregar os dados
dataset = pd.read_csv('../datasets/dataset.csv')

# Pré-Processamento dos Dados
dataset = pd.get_dummies(dataset, columns=['tipo', 'caixa', 'combustivel', 'marca', 'com_danos'])

# Lidar com possíveis valores nulos
dataset = dataset.dropna()

# Separar features (X) e target (y)
X = dataset.drop(['id', 'preco'], axis=1)
Y = dataset['preco']

In [6]:
# Modelo Random Forest
rf_model = RandomForestRegressor()

# Treinar o modelo com todos os dados
rf_model.fit(X, Y)

RandomForestRegressor()

In [7]:
# Estimar os preços para as viaturas com características no just_features.csv
just_features = pd.read_csv('../datasets/just_features.csv')

# Garantir que as features são as mesmas usadas para treino do modelo
just_features_processed = pd.get_dummies(just_features, columns=['tipo', 'caixa', 'combustivel', 'marca', 'com_danos'])
just_features_processed = just_features_processed.reindex(columns=X.columns, fill_value=0)

# Adicionar a coluna 'id' de volta ao just_features_processed
just_features_processed['id'] = just_features['id']

# Fazer as estimativas
estimated_prices = rf_model.predict(just_features_processed.drop('id', axis=1))

estimated_prices_rounded = estimated_prices.round(0)

# Criar DataFrame com as estimativas arredondadas
results_df = pd.DataFrame({'id': just_features_processed['id'], 'preco': estimated_prices_rounded.astype(int)})

# Salvar as estimativas arredondadas em precos_estimados.csv
results_df.to_csv('../datasets/precos_estimados.csv', index=False)

In [8]:
# Avaliar o modelo no conjunto de teste
y_pred_all = rf_model.predict(X)
r2_all = r2_score(Y, y_pred_all)
print(f"R² no conjunto completo: {r2_all}")

# Calcular R² para as estimativas
r2_estimates = r2_score(Y[:len(estimated_prices)], estimated_prices)
print(f"R² para as estimativas no conjunto completo: {r2_estimates}")

R² no conjunto completo: 0.9611688328384665
R² para as estimativas no conjunto completo: -0.1814089110521342
